In [ ]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
from common.utils import seed_everything
from train import get_dataset
from draw_utils import visualize_dataset

In [ ]:
from common.logger import logger
import logging

# DEBUG INFO WARNING ERROR CRITICAL
logger.setLevel(logging.INFO)

In [ ]:
seed_everything(1024)

# Backbone

In [ ]:
import torch
from torch import nn

from torchvision.models.detection.backbone_utils import resnet_fpn_backbone

In [ ]:
backbone = resnet_fpn_backbone(
    'resnet18', # resnet18, resnet50
    pretrained=True,
    trainable_layers=5, # all layers
    # trainable_layers=3,
    returned_layers=[2,3,4]
)

In [ ]:
test_tensor = torch.randn(1,3,300,300)
feature_dict = backbone(test_tensor)
feature_dict.keys()

In [ ]:
features = list(feature_dict.values())

In [ ]:
for f in features:
    print(f.shape)

# PennFudanPed

In [ ]:
from dataset.penn_fudan_dataset import PennFudanDataset

In [ ]:
PENN_FUDAT_ROOT_PATH = './data/PennFudanPed'
dataset_train = PennFudanDataset(PENN_FUDAT_ROOT_PATH, train=True)

In [ ]:
img, mask, bboxs = dataset_train[0]
img.shape, mask.shape, bboxs.shape

In [ ]:
visualize_dataset(dataset_train, count=5, size=3)

# Pascal VOC

In [ ]:
from dataset.pascal_voc_dataset import PascalVOCDataset

In [ ]:
dataset_voc = PascalVOCDataset('./data/VOCdevkit/', 'TRAIN', transforms=None)

In [ ]:
visualize_dataset(dataset_voc, count=5, size=3)

# Model

In [ ]:
from centernet import create_model

In [ ]:
model = create_model()

In [ ]:
img, mask, bboxs = dataset_train[0]
img.shape, mask.shape, bboxs.shape

In [ ]:
out = model(img)

In [ ]:
out.shape

# Test

In [ ]:
dataset_train, dataset_test = get_dataset()

In [ ]:
model_name = 'centernet_v1'
model_save_name = f'{model_name}.pth'
model.load_state_dict(torch.load(model_save_name))
logger.info(f'Model loaded from {model_save_name}')

In [ ]:
import numpy as np
import torchvision.transforms as T
import cv2

from draw_utils import show_images

In [ ]:
def make_prediction(model, dataset, index):
    test_img_t, test_mask_t, test_bboxs_t = dataset[index]

    print(test_img_t.shape)

    model.eval()
    pred = model(test_img_t)

    print(pred.shape)

    pred_mask = pred[:,0]
    # pred_mask = torch.sigmoid(pred[:,0])
    bboxs = pred[:,1:]

    pred_img = pred_mask.squeeze().detach().cpu().numpy()

    # threshold = 15
    threshold = 0
    print(np.min(pred_img), np.max(pred_img))
    pred_img = np.where(pred_img > threshold, pred_img, 0)

    # draw bbox
    color = (255, 0, 0)
    img_arr = np.array(T.ToPILImage()(test_img_t))
    xs, ys = np.nonzero(pred_img)
    bboxs = bboxs.squeeze().detach().cpu().numpy()
    bboxs = np.transpose(bboxs, (1, 2, 0))
    center_bboxs = np.int32(bboxs[xs, ys])
    for bbox in center_bboxs:
        # print(bbox)
        cv2.rectangle(img_arr, (bbox[0],bbox[1]), (bbox[2],bbox[3]), color, 2)
    
    # visualize
    show_images([img_arr, test_mask_t, pred_img],
                ["Original", "GT Mask", "Prediction"])

In [ ]:
make_prediction(model, dataset_test, index=4)